## Assignment 5B

### Problem 1 (100 Points)
Anharmonic Oscillator. Consider the Schrodinger equation for a particle of mass m in the anharmonic potential $V(x)=V_0\tfrac{x^4}{a^4}$:
$$-\dfrac{1}{2}\dfrac{d^2\psi(x)}{dx^2}+V(x)\psi(x)=E\psi(x)$$
where $V_0$ and $a$ are constants. For simplicity, in this exercise set $\tfrac{V_0}{a^4}=\tfrac{1}{2}$.

After converting this second-order equation into a 2-dimensional first-order equation, write a program to solve this system using the shooting method and the 4th-order Runge-Kutta (RK4) method.

As explained in class, you will need to write a program to find the energies, starting with an initial guess for the energy and using a root-finding method to refine the guess until you find accurate values.

We can write the differential equation as:
$$\begin{align}
-\dfrac{1}{2}\dfrac{d^2\psi(x)}{dx^2}+\dfrac{1}{2}x^4\psi(x)&=E\psi(x)\\
\dfrac{d^2\psi(x)}{dx^2}-x^4\psi(x)&=-2E\psi(x)\\
\dfrac{d^2\psi(x)}{dx^2}&=\left(x^4-2E\right)\psi(x)\\
\end{align}$$
Converting into 2 differential equations, we get:
$$\begin{align}
\dfrac{d\psi(x)}{dx}&=\phi(x)\\
\dfrac{d\phi(x)}{dx}&=\left(x^4-2E\right)\psi(x)\\
\end{align}$$

#### Problem 1.A (30 Points)
Use Simpson’s method to calculate the relevant numerical integral in the WKB
formula:
$$B=\int_{-1}^1\sqrt{1-s^4}\ ds$$
and obtain the corresponding WKB approximations to the eigenvalues:
$$E_n\sim\dfrac{1}{2}\left(\pi\dfrac{n+\tfrac{1}{2}}{B}\right)^\tfrac{4}{3}$$
**Hint:** Note that the integrand of $B$ is relatively steep near the integration boundaries. Make sure to use a sufficiently tight mesh to have at least $4$ significant digits of accuracy in the eigenvalues, for $n = 0, 1, ..., 6$.

In [3]:
import numpy as np

def simpson(f: callable,
            a: float|int,
            b: float|int, *,
            dx: float|int|None = None,
            N: int|None = None
            ) -> float:
    # Function requirements
    assert callable(f)
    assert isinstance(a, (int, float))
    assert isinstance(b, (int, float))
    assert N is not None or dx is not None
    assert not (N is not None and dx is not None)
    if N is not None: assert isinstance(N, int)
    if dx is not None: assert isinstance(dx, (int, float))

    # Calculate the integral
    if N is not None:
        x = np.linspace(a, b, N+1)
        return np.sum(f(x[:-2:2]) + 4*f(x[1:-1:2]) + f(x[2::2])) * (b-a)/(3*N)
    elif dx is not None:
        x = np.arange(a, b+dx, dx)
        return np.sum(f(x[:-2:2]) + 4*f(x[1:-1:2]) + f(x[2::2])) * dx/3
    else:
        raise Exception("Something went wrong...")

def WKB_energy(n: int) -> float:
    B = simpson(lambda x: np.sqrt(1-x**4), -1, 1, N=1_000_000)
    internal = np.pi*(n + 1/2)/B
    return (1/2) * internal**(4/3)

B = simpson(lambda x: np.sqrt(1-x**4), -1, 1, N=1_000_000)
print(f"{B=:.10f}")
print(f"{WKB_energy(0)=:.10f}")
print(f"{WKB_energy(1)=:.10f}")
print(f"{WKB_energy(2)=:.10f}")
print(f"{WKB_energy(3)=:.10f}")
print(f"{WKB_energy(4)=:.10f}")
print(f"{WKB_energy(5)=:.10f}")
print(f"{WKB_energy(6)=:.10f}")

B=1.7480383686
WKB_energy(0)=0.4335726635
WKB_energy(1)=1.8759599631
WKB_energy(2)=3.7069941290
WKB_energy(3)=5.8057626767
WKB_energy(4)=8.1168073520
WKB_energy(5)=10.6068266870
WKB_energy(6)=13.2531677648


#### Problem 1.B (50 Points)
Start from one of the boundaries—in a similar fashion as in the example of the
infinite square well discussed in class. Calculate the eigenfunctions and the eigenvalues of the problem for $n = 0, 1, ..., 6$ (for simplicity, you are not required to normalize the wavefunctions in this exercise).

#### Problem 1.C (20 Points)
Compare your numerical energy eigenvalues with the WKB energies and plot the
results as a function of the excitation number $n$. Discuss the agreement as a function of $n$.